In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import os
import numpy as np
import pandas as pd

base_dir='/content/gdrive/My Drive/빅데이터 공모전/빅콘테스트 데이터/제공데이터'
sales_data='19년도-실적데이터.xlsx'
sales_dir=os.path.join(base_dir, sales_data)

# [ 추정에 불필요한 데이터 제거 및 전처리 ]

In [ ]:
df_sales=pd.read_excel(sales_dir,
                       names=['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액'],
                       na_values='NaN')

In [ ]:
df_sales.dtypes

방송일시     object
노출(분)    object
마더코드     object
상품코드     object
상품명      object
상품군      object
판매단가     object
취급액      object
dtype: object

In [ ]:
df_sales

In [ ]:
df_sales=df_sales.drop(index=0)
df_sales_NaN=df_sales[df_sales['판매단가']==0] #과거 실적이 없는 경우에 대해선 유사 카테고리 또는 동일머더코드로 예측
df_sales=df_sales.drop(df_sales[(df_sales['판매단가'] == 0)].index) #'판매단가' 0원인 것은 추정에서 제외시킨다.

In [ ]:
df_sales.방송일시=pd.to_datetime(df_sales.방송일시, format='%Y%m%d %H:%M:%S')
df_sales['month']=df_sales.방송일시.dt.strftime('%m')
df_sales['week day']=df_sales['방송일시'].dt.day_name()
df_sales['broad date']=df_sales['방송일시'].dt.date
df_sales['broad time']=df_sales['방송일시'].dt.time
df_sales=df_sales.drop('방송일시', axis=1)

In [ ]:
df_sales

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,month,week day,broad date,broad time
1,20,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000,01,Tuesday,2019-01-01,06:00:00
2,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000,01,Tuesday,2019-01-01,06:00:00
3,20,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000,01,Tuesday,2019-01-01,06:20:00
4,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000,01,Tuesday,2019-01-01,06:20:00
5,20,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000,01,Tuesday,2019-01-01,06:40:00
...,...,...,...,...,...,...,...,...,...,...,...
38300,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000,12,Tuesday,2019-12-31,23:40:00
38301,20,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000,01,Wednesday,2020-01-01,00:00:00
38302,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000,01,Wednesday,2020-01-01,00:00:00
38303,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000,01,Wednesday,2020-01-01,00:00:00


In [ ]:
df_sales=df_sales.drop(df_sales[(df_sales['week day'] == 'Saturday')].index)

In [ ]:
df_sales['broad time']

1        06:00:00
2        06:00:00
3        06:20:00
4        06:20:00
5        06:40:00
           ...   
38300    23:40:00
38301    00:00:00
38302    00:00:00
38303    00:00:00
38304    00:00:00
Name: broad time, Length: 32190, dtype: object

In [ ]:
df_sales=df_sales.reset_index(drop=True)

In [ ]:
#토요일 18:00-18:20 추정에서 제외(토요일은 제외시켰음)
#심야시간대 추정에서 제외

In [ ]:
df_sales

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,month,week day,broad date,broad time
0,20,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000,01,Tuesday,2019-01-01,06:00:00
1,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000,01,Tuesday,2019-01-01,06:00:00
2,20,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000,01,Tuesday,2019-01-01,06:20:00
3,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000,01,Tuesday,2019-01-01,06:20:00
4,20,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000,01,Tuesday,2019-01-01,06:40:00
...,...,...,...,...,...,...,...,...,...,...,...
32185,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000,12,Tuesday,2019-12-31,23:40:00
32186,20,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000,01,Wednesday,2020-01-01,00:00:00
32187,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000,01,Wednesday,2020-01-01,00:00:00
32188,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000,01,Wednesday,2020-01-01,00:00:00


In [ ]:
6751



## *   과거 실적이 없는 경우 분석



In [ ]:
df_sales_NaN

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
2319,2019-01-25 19:00:00,60,100507,201567,TCOM_처브치아보험,무형,0,NaN
2846,2019-01-31 18:00:00,60,100507,201567,TCOM_처브치아보험,무형,0,NaN
3333,2019-02-05 18:00:00,60,100507,201567,TCOM_처브치아보험,무형,0,NaN
6571,2019-03-09 00:00:00,20,100324,201095,온라인투어 동남아 나트랑/빈펄랜드 (190309),무형,0,NaN
6572,2019-03-09 00:20:00,20,100324,201095,온라인투어 동남아 나트랑/빈펄랜드 (190309),무형,0,NaN
...,...,...,...,...,...,...,...,...
38305,2020-01-01 00:20:00,20,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38306,2020-01-01 00:40:00,20,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38307,2020-01-01 01:00:00,20,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38308,2020-01-01 01:20:00,20,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN


In [ ]:
df_sales_NaN.상품명.unique()

array(['TCOM_처브치아보험', '온라인투어 동남아 나트랑/빈펄랜드 (190309)',
       '(무)메리츠 올바른 간편보험1810', '하나투어 괌PIC(190315)', '온라인투어 오키나와 (190323)',
       '온라인투어 중국/장사,장가계 (190330)', '온라인투어 오사카(0406)', '넥센타이어 렌탈서비스',
       '롯데렌터카', '하나투어 북해도(190413)', '하나투어 오키나와(190420)',
       '온라인투어 장사장가계(190420)', '온라인투어 동남아 다낭5일(0427)',
       '온라인투어 호주/뉴질랜드(190427)', '넥센타이어 렌탈서비스(특약)', 'TCOM_아이러너 런닝머신 렌탈',
       '웰릭스렌탈 음식물처리기', '여행박사 대만(190503)', '온라인투어 하와이 6일(190504)',
       '온라인투어 북해도(190511)', 'TCOM_현대해상 (무)하이콜운전자상해보험',
       '하나투어 오사카 (190518)', '온라인투어 태항산(0518)', ' 용인공원라이프 스마트라이프48',
       '온라인투어 하이난(0601)', '온라인투어 동경/오다이바/하코네 (0601)',
       '온라인투어 미동부/캐나다 (190608)', '온라인투어 청도(190615)',
       '온라인투어 호주/오클랜드(190622)', '롯데JTB 태항산5일(190628)', '온라인투어 북경(190706)',
       'DB통큰간편암플랜', 'TCOM_관절염 치료 의료기기 닥터88', 'SK매직정수기 3종',
       '롯데JTB 장가계(190726)', '롯데JTB 장가계(190803)', '온라인투어 태항산(190817)',
       '온라인투어 태항산(190823) 상담예약', '참좋은여행 장가계(190823) 상담예약',
       'TCOM_바디프랜드 아제라', '온라인투어 장가계 항공직항(190906) 상담예약',
      

In [ ]:
for products in df_sales_NaN.상품명.unique():
  if 